## Text Generation using LSTMs

#### In this program we are using only 1 batch and our sequence length is same(Although for making a single batch is a bad idea but we have to analyze the LSTMs first with a single batch size )

### Here we use a character wise computation in a sequence means we are going to generate a future character based on past results(not word wise)

In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import datetime

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

In [155]:
ds = pd.read_csv('/home/chintan/Dataset/Markov rrelated/trump_tw.csv')
#ds.head()    #Guve dataframe dataset alongwith class label (5,64) 64 columns it have
ds.text.head() #Now give the rows in which t
print len(ds.text[0])

112


In [3]:
#ds.text give all rows in our dataset(means all tweets)

In [109]:
# Take all the text together(Clean the data)

data = ' '.join([ix for ix in ds.text]) #Join all the tweets with space
print data[:500]

Read a great interview with Donald Trump that appeared in The New York Times Magazine: http://tinyurl.com/qsx4o6 Congratulations to Evan Lysacek for being nominated SI sportsman of the year. He's a great guy, and he has my vote!  #EvanForSI I was on The View this morning. We talked about The Apprentice. Tonight's episode is a great one--tough, exciting and surprising. 10 pm/NBC Tomorrow night's episode of The Apprentice delivers excitement at QVC along with appearances by Isaac Mizrahi and Cathi


In [5]:
print set(data)       #This set now gives the uniques keys
print len(set(data))  #There are 174 unique different keys in our data of tweets
#These \x83 are not an Ascii value

set(['\x83', '\x87', '\x8b', '\x8f', '\x93', '\x97', '\x9b', '\x9f', ' ', '\xa3', '$', '\xa7', '(', '\xab', ',', '\xaf', '0', '\xb3', '4', '\xb7', '8', '\xbb', '\xbf', '@', '\xc3', 'D', 'H', 'L', 'P', 'T', '\xd7', 'X', '\\', '`', '\xe3', 'd', '\xe7', 'h', 'l', '\xef', 'p', 't', 'x', '|', '\x80', '\x84', '\x88', '\x8c', '\x90', '\x94', '\x98', '\x9c', '\xa0', '#', '\xa4', "'", '\xa8', '+', '\xac', '/', '\xb0', '3', '\xb4', '7', '\xb8', ';', '\xbc', '?', 'C', '\xc4', 'G', 'K', 'O', 'S', 'W', '[', '_', 'c', '\xe4', 'g', '\xe8', 'k', 'o', '\xf0', 's', '\xf4', 'w', '{', '\x81', '\x85', '\x89', '\n', '\x8d', '\x91', '\x95', '\x99', '\x9d', '\xa1', '"', '\xa5', '&', '\xa9', '*', '\xad', '.', '\xb1', '2', '\xb5', '6', '\xb9', ':', '\xbd', 'B', '\xc5', 'F', 'J', 'N', 'R', 'V', 'Z', '\xe1', 'b', '\xe5', 'f', '\xe9', 'j', 'n', 'r', 'v', 'z', '~', '\x82', '\x86', '\x8a', '\r', '\x8e', '\x92', '\x96', '\x9a', '\x9e', '!', '\xa2', '%', '\xa6', ')', '\xaa', '-', '\xae', '1', '\xb2', '5', '9', '\xba',

In [6]:
# Create Vocab
vocab = list(set(data))
#print vocab               #Vocab is now the list of all unique keys

i2c, c2i = {}, {}          # We construct tow empty dictionaries integertochaararcter and character to integer
                           #Basically two tables at a particular index value return the corresponding label value

for idx, chx in enumerate(vocab):  #Now enumerate over all keys value enumerate return two things first value 
    #print idx,chx               #  is the index value and the other one is respective character at particular index
    i2c[idx] = chx             # Har index ke corresponding uska character
    c2i[chx] = idx             # Harcharacter ke corresponding uski index value
    #print i2c[idx],c2i[chx]

#print len(c2i.keys())             #All 174 keys

In [7]:
def get_onehot(x):
    # Take input a string and convert to one-hot encoding
    vec_size = len(c2i.keys())         # vec_size= 174 for now
    n_seq = len(x)
    data = np.zeros((1, n_seq, vec_size))      # we make a zeroes matrix of 1 batch and n_seq length of sequence 
                                               # And 174 vec_size unique characters
    # For each element in the list
    for ix in range(n_seq):              #Now we are maikng an one hot vector for ecah character in a sequence 
        curr_char = x[ix]                 #curr_charcter in sequence
        oh_index = c2i[curr_char]        # us curr_chaarcter ki index value 
        #print ix, curr_char, oh_index
        data[:, ix, oh_index] = 1         #us index value pr 1 krna h jsse one hot milega
    return data
#print data[:1000]
#print get_onehot('this is my string')  #Thus is matrix of one hot vector           
#One hot vector in form of(1,length,vec_size) mean(1=batchsize,lenght=sque lenght in that case it is 17, vect size=174 unique characters)
#print get_onehot('this is my string').shape      #(1,17,174)  #this string is of 17 length character and corrsponding we find
print get_onehot('this is my string').argmax(2)    #this string is of 17 length character and corrsponding we find
                                                  #the argmax means the index value of charcater 
# means t is at 41 index out of 174 thats why we give here argmax(axis=2)

[[ 41  37 168  84   8 168  84   8 169 172   8  84  41 127 168 126  79]]


In [8]:
for ix in ds.text[:10]:
    print get_onehot(ix).shape
    #print get_onehot(ix).argmax(2) #this will give for each character its label value at which one ot make 1
#print get_onehot('t')       #See this is one hot vector for letter t and the one hot means 1 is on at index value 41

(1, 112, 174)
(1, 127, 174)
(1, 139, 174)
(1, 140, 174)
(1, 116, 174)
(1, 122, 174)
(1, 108, 174)
(1, 117, 174)
(1, 115, 174)
(1, 102, 174)


#### Now LSTMs

In [9]:
#nn.LSTM?

In [167]:
class CharNN(nn.Module):
    def __init__(self, in_shape=None, out_shape=None, hidden_shape=None):
        super(CharNN, self).__init__()     #basic lstm format
        self.in_shape = in_shape
        self.out_shape = out_shape
        self.hidden_shape = hidden_shape
        self.n_layers = 1                 # NO. of Lstm layers
        
        self.rnn = nn.LSTM(
            input_size=self.in_shape,
            hidden_size=self.hidden_shape,
            num_layers=self.n_layers,
            batch_first=True               #batch_first true means we want the shape is(batch_size,......) means shape start from batch_size
        )
        self.out = nn.Linear(self.hidden_shape, self.out_shape)  #means wh vector is hidden_shape(256) and out_shape is out 174 same length as input
    
    def forward(self, x, h):                 #Note here h_state is not none we have to find also h_state
        r_out, h_state = self.rnn(x, h)        #r_out is output at each timestamp # r_out (batch, time_step, hidden_size)
        
        outs = []
        for ix in range(r_out.size(1)):     #for each time_stamp we are appending the out list
            current_out = F.softmax(self.out(r_out[:, ix, :])) 
            outs.append(current_out)    # in out probabities 
            #print outs[0]
        return torch.stack(outs, dim=1), h_state
    
    def predict(self, char, h=None, top_k=None):  #top_k is used to take top k probabilities (like in markov we use temp to make all character possibilties equal) similarly take random k probabilites each time
        if h is None:
            h = self.init_hidden(1, gpu=False)
        
        x = get_onehot(char) #in out out has probabilities corresponding to each character.
        out, h = self.forward(torch.FloatTensor(x), h)       # x is onehotvector forward rnn and find corresponding future character
        print out.shape #,h              #out.shape=(1,1,174) and h is tuple
        p = out.data
        #p is our next charcter probablities
        print '-'*10
        print p.shape
        if top_k is None:
            top_ch = np.arange(self.out_shape)    #arrange the out array
        else:
            p, top_ch = p.topk(top_k) #p.topk is predefined method in pytorch that can find return the previous indices(top_ch) and the top k probabilities 
            #print p.shape                ##torch.Size([1, 1, 100]) 
            #print '-'*10
            #p.topk?
            #print top_ch.shape          #torch.Size([1, 1, 100]) 
            top_ch = top_ch.numpy().squeeze()  #means remove 1 out of shape 
        
        p = p.numpy().squeeze()
        #print p.shape  #now shape is (100,)
        char = np.random.choice(top_ch, p=p/p.sum())  #char is here an interger value for now
        return i2c[char], h                  #return an integer index value of char
    
    #Here we are going to compute the initial state
    def init_hidden(self, batch_size, gpu=False):  #Note this h should be a tuple because we have to store two things
        if gpu:                                    #one is current context(ht-1) and other one is past context(Ct)
            return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_shape).cuda()),
                    Variable(torch.zeros(self.n_layers, batch_size, self.hidden_shape)).cuda())
        return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_shape)),      #return a tuple(ht-1,Ct)
                Variable(torch.zeros(self.n_layers, batch_size, self.hidden_shape)))

In [168]:
model = CharNN(in_shape=174, out_shape=174, hidden_shape=256)
#model.cuda()
print model

CharNN(
  (rnn): LSTM(174, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=174, bias=True)
)


In [169]:
# Load the weights
#model.load_state_dict(torch.load('/home/shubham/all_projects/CB/Summer_2018/data/checkpoints/text_gen/model_256h_epoch_38.ckpt'))

In [170]:
# model.predict('a', top_k=20)[0]

In [171]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [150]:
#model.state_dict?

In [166]:
# Set to train mode
# model.cuda()
model.train()      #because dropout layer is used so bring it on traininh mode
N = 15              #N should be 200000 for better results

for epoch in range(5):
    total_loss = 0
    # For each sequence
    for qx in range(N):
        seqx = ds.text[qx]
        h_state = model.init_hidden(1)
        input_seq = seqx[:-1]      # Input sequence is our one length kam because like ababc for abab we find babc so we dont need last c at training time only
        target_seq = seqx[1:]   # Target will start from 1 onwards
        
        x = Variable(torch.FloatTensor(get_onehot(input_seq)), requires_grad=True)# .cuda()
        y = Variable(torch.LongTensor(get_onehot(target_seq).argmax(2)))# .cuda()
        print y
        model.zero_grad()
        pred, h_state = model.forward(x, h_state)
        print pred.squeeze().shape, y.shape         #torch.Size([111, 174]) torch.Size([1, 111])
        loss = criterion(pred.squeeze(), y.squeeze()) #loss function
        # optimizer.zero_grad()
        loss.backward()
        
        # gradient clipping to solve exploding/vanishing grads
        # clip = 5.0
        # nn.utils.clip_grad_norm(net.parameters(), clip)
        
        optimizer.step()
        total_loss += loss
        if qx%(N/5) == 0:
            print 'Loss: {} at Epoch: {} | Seq: {}'.format(loss, epoch, qx)
        
    print "Overall Average Loss: {} at Epoch: {}".format(total_loss / float(N), epoch)
    
    # Save model checkpoints
    if epoch % 10 == 0:
        torch.save(model.state_dict(), "./data/checkpoints/text_gen/model_256h_epoch_{}.ckpt".format(epoch))

tensor([[ 166,  164,   35,    8,  164,    8,   79,  127,  166,  164,
           41,    8,  168,  126,   41,  166,  127,  128,  168,  166,
           86,    8,   86,  168,   41,   37,    8,   25,   82,  126,
          164,   38,   35,    8,   29,  127,  171,  169,   40,    8,
           41,   37,  164,   41,    8,  164,   40,   40,  166,  164,
          127,  166,   35,    8,  168,  126,    8,   29,   37,  166,
            8,  116,  166,   86,    8,  162,   82,  127,   81,    8,
           29,  168,  169,  166,   84,    8,  159,  164,   79,  164,
          129,  168,  126,  166,  110,    8,   37,   41,   41,   40,
          110,   59,   59,   41,  168,  126,  172,  171,  127,   38,
          104,   77,   82,  169,   59,  170,   84,   42,   18,   82,
          108]])
torch.Size([111, 174]) torch.Size([1, 111])


/home/chintan/.local/lib/python2.7/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Loss: 5.03929328918 at Epoch: 0 | Seq: 0
tensor([[  82,  126,   79,  127,  164,   41,  171,   38,  164,   41,
          168,   82,  126,   84,    8,   41,   82,    8,  157,  128,
          164,  126,    8,   27,  172,   84,  164,   77,  166,   81,
            8,  123,   82,  127,    8,  121,  166,  168,  126,   79,
            8,  126,   82,  169,  168,  126,  164,   41,  166,   35,
            8,   73,  158,    8,   84,   40,   82,  127,   41,   84,
          169,  164,  126,    8,   82,  123,    8,   41,   37,  166,
            8,  172,  166,  164,  127,  104,    8,   26,  166,   55,
           84,    8,  164,    8,   79,  127,  166,  164,   41,    8,
           79,  171,  172,   14,    8,  164,  126,   35,    8,   37,
          166,    8,   37,  164,   84,    8,  169,  172,    8,  128,
           82,   41,  166,  140,    8,    8,   53,  157,  128,  164,
          126,  114,   82,  127,   73,  158]])
torch.Size([126, 174]) torch.Size([1, 126])
tensor([[   8,   86,  164,   84,    8,  

tensor([[ 126,   41,  166,  127,    8,   41,   37,  166,    8,   98,
           29,   37,  168,  126,   81,    8,   27,  168,   81,  166,
            8,  155,    8,   68,   37,  164,  169,   40,  168,   82,
          126,   98,    8,   84,  168,   79,  126,  166,   35,    8,
          121,   82,   82,   81,    8,  164,  126,   35,    8,   81,
          166,  172,   77,   37,  164,  168,  126,    8,   77,   82,
          126,   41,  166,   84,   41,  110,   91,   37,   41,   41,
           40,  110,   59,   59,   86,   86,   86,  104,   41,  127,
          171,  169,   40,   41,   37,  168,  126,   81,   38,  168,
           81,  166,  164,   77,   37,  164,  169,   40,  168,   82,
          126,  104,   77,   82,  169,   59,   77,   82,  126,   41,
          166,   84,   41,   59]])
torch.Size([114, 174]) torch.Size([1, 114])
tensor([[  82,  126,   79,  127,  164,   41,  171,   38,  164,   41,
          168,   82,  126,   84,    8,   41,   82,    8,  112,  127,
          164,  126,   3

KeyboardInterrupt: 

In [172]:
sentence = 'o'
model.cpu()
h_s = model.init_hidden(1, gpu=False)
for ix in range(1000):
    char = sentence[-1]
    out, h = model.predict(char, h=h_s, top_k=100)
    h_s = (h[0].data, h[1].data)
    
    sentence += out
print sentence

/home/chintan/.local/lib/python2.7/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174

torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174

torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174

torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174

torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174])
torch.Size([1, 1, 174])
----------
torch.Size([1, 1, 174